# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [16]:
# Import semua libraray
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Memuat dataset
df = pd.read_csv('hasil_clustering.csv')
df.head()

,status_type,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,Cluster
0,1,2018-04-21,150,0,0,150,0,0,0,0,0,1
1,1,2018-04-21,111,0,0,111,0,0,0,0,0,1
2,3,2018-03-20,18,0,0,18,0,0,0,0,0,0
3,3,2018-03-19,37,0,1,37,0,0,0,0,0,2
4,3,2018-01-19,35,1,0,35,0,0,0,0,0,2


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3123 entries, 0 to 3122
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   status_type    3123 non-null   float64
 1   num_reactions  3123 non-null   float64
 2   num_comments   3123 non-null   float64
 3   num_shares     3123 non-null   float64
 4   num_likes      3123 non-null   float64
 5   num_loves      3123 non-null   float64
 6   num_wows       3123 non-null   float64
 7   num_hahas      3123 non-null   float64
 8   num_sads       3123 non-null   float64
 9   num_angrys     3123 non-null   float64
 10  Cluster        3123 non-null   int64  
dtypes: float64(10), int64(1)
memory usage: 268.5 KB


In [3]:
# Drop kolom status_published
df = df.drop('status_published', axis=1)

In [4]:
# Periksa kolom target (contoh: Cluster)
print(df['Cluster'].unique())
print(df['Cluster'].dtype)

[1 0 2]
int64


In [5]:
# Konversi kolom 'Cluster' menjadi integer kategori
df['Cluster'] = df['Cluster'].astype(int)

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [6]:
# Buat instance MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi hanya kolom numerik kecuali target
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.drop('Cluster')
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['Cluster'])
y = df['Cluster']

# Split data menjadi set pelatihan dan set uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(2498, 10), y_train=(2498,)
Test set shape: X_test=(625, 10), y_test=(625,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [7]:
# Mendefinisikan setiap klasifikasi secara terpisah
knn = KNeighborsClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
svm = SVC().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

Mencoba semua algoritma yang digunakan pada teknik supervised learning untuk menentukan model terbaik yang dapat digunakan pada data/kasus ini.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [23]:
# Fungsi untuk mengevaluasi dan mengembalikan hasil sebagai dictionary
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    results = {
        'Confusion Matrix': cm,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision (macro)': precision_score(y_test, y_pred, average='macro'),
        'Recall (macro)': recall_score(y_test, y_pred, average='macro'),
        'F1-Score (macro)': f1_score(y_test, y_pred, average='macro')
    }
    return results

# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(dt, X_test, y_test),
    'Random Forest (RF)': evaluate_model(rf, X_test, y_test),
    'Support Vector Machine (SVM)': evaluate_model(svm, X_test, y_test),
    'Naive Bayes (NB)': evaluate_model(nb, X_test, y_test)
}

# Buat DataFrame untuk meringkas hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision (macro)'],
        'Recall': metrics['Recall (macro)'],
        'F1-Score': metrics['F1-Score (macro)']
    })

# Konversi ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

                          Model  Accuracy  Precision    Recall  F1-Score
0     K-Nearest Neighbors (KNN)    0.9776   0.956543  0.940840  0.948322
1            Decision Tree (DT)    1.0000   1.000000  1.000000  1.000000
2            Random Forest (RF)    1.0000   1.000000  1.000000  1.000000
3  Support Vector Machine (SVM)    0.9824   0.988537  0.965448  0.976610
4              Naive Bayes (NB)    0.9632   0.918239  0.970774  0.942192


model Decision Tree dan Random Forest memiliki performa yang baik, namun untuk semua model yang diterapkan memiliki performa yang sangat baik pada kasus/datset ini.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [9]:
#Type your code here

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [10]:
#Type your code here

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

1. K-Nearest Neighbors (KNN)
  Accuracy: 0.9776
  Precision: 0.9565
  Recall: 0.9408
  F1-Score: 0.9483

  KNN menunjukkan performa yang baik dengan akurasi tinggi dan nilai F1-Score yang cukup baik, tetapi recall yang sedikit lebih rendah dibandingkan dengan precision. Ini bisa mengindikasikan adanya masalah dengan False Negative (FN), di mana model cenderung lebih sering salah mengklasifikasikan beberapa data sebagai negatif padahal sebenarnya positif. Model mungkin kurang sensitif terhadap kelas positif minoritas.
2. Decision Tree (DT)
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000

  Decision Tree menunjukkan hasil sempurna dengan akurasi, precision, recall, dan F1-Score semuanya mencapai 1. Namun, ini dapat menunjukkan adanya overfitting, karena keputusan model mungkin hanya sangat cocok dengan data pelatihan dan tidak mampu generalisasi dengan baik untuk data uji. Biasanya, model ini rentan terhadap overfitting jika tidak ada pembatasan pada kedalaman pohon atau jumlah contoh minimum per node.
3. Random Forest (RF)
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000

  Random Forest juga menunjukkan hasil sempurna, tetapi sama seperti Decision Tree, ini bisa menjadi indikasi overfitting. Random Forest menggabungkan beberapa decision tree, namun tanpa parameter yang tepat, model bisa sangat cocok dengan data pelatihan dan tidak dapat bekerja dengan baik pada data uji.
4. Support Vector Machine (SVM)
  Accuracy: 0.9824
  Precision: 0.9885
  Recall: 0.9654
  F1-Score: 0.9766

  SVM menunjukkan performa yang sangat baik, dengan precision yang sangat tinggi dan recall yang juga cukup baik. Recall sedikit lebih rendah, yang mungkin menunjukkan bahwa model sedikit lebih cenderung mengklasifikasikan beberapa kelas positif sebagai negatif. Namun, secara keseluruhan, model ini seimbang dan memiliki performa yang solid.
5. Naive Bayes (NB)
  Accuracy: 0.9632
  Precision: 0.9182
  Recall: 0.9708
  F1-Score: 0.9422

  Naive Bayes menunjukkan hasil yang lebih rendah dalam hal precision, meskipun recall-nya cukup tinggi. Hal ini menunjukkan bahwa model cenderung mengklasifikasikan lebih banyak kelas positif (mengorbankan beberapa kesalahan tipe I). Precision yang lebih rendah dapat disebabkan oleh False Positive (FP) yang tinggi.

- **Kesimpulan dan Rekomendasi Tindakan Lanjutan**
- **Overfitting**
  Model seperti Decision Tree dan Random Forest menunjukkan hasil yang sempurna pada data uji, yang sering kali menjadi indikasi overfitting. Untuk memperbaikinya, Anda dapat mencoba teknik regularisasi atau mengoptimalkan parameter untuk mengurangi kompleksitas model.
- **Precision dan Recall**
  KNN memiliki recall yang sedikit lebih rendah dibandingkan precision, yang menunjukkan adanya potensi masalah pada kelas minoritas. Menggunakan teknik seperti class weighting atau sampling techniques dapat membantu mengatasi masalah ini.
- **Model yang Solid**
  SVM menunjukkan hasil yang sangat baik secara keseluruhan, dengan precision dan recall yang seimbang. Model ini bisa menjadi pilihan yang solid, terutama jika Anda menghadapi masalah klasifikasi yang kompleks.